In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import io
import sys
import os
from azure.identity import AzureCliCredential
from azure.storage.filedatalake import DataLakeServiceClient, DataLakeFileClient
from ast import literal_eval

In [2]:
# Must run this code before we can write any files. Only needs to be ran once per python script.
cred = AzureCliCredential() 
storage_account = "datalakestoragenfsk522vz" 
file_system = "datalakegen2filesystem" 

service_client = DataLakeServiceClient(
    f"https://{storage_account}.dfs.core.windows.net/", credential=cred
)

In [3]:
new_file_client = service_client.get_file_client(file_system, "analytics/auto/commercial/00001_TestConnectivity/iris.csv")
new_file_client.exists() # Checking if file aleady exists

True

In [4]:
test_iris = pd.read_csv(io.BytesIO(new_file_client.download_file().readall()))
test_iris.head()

,id,sepal_length,sepal_width,petal_length,petal_width,species
0,1,5.1,3.5,1.4,0.2,setosa
1,2,4.9,3.0,1.4,0.2,setosa
2,3,4.7,3.2,1.3,0.2,setosa
3,4,4.6,3.1,1.5,0.2,setosa
4,5,5.0,3.6,1.4,0.2,setosa
